In [2]:
import os
import numpy as np
import tensorflow as tf


def load_minibatch(index, var):
    shape = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_shape.npy'))
    for i in range(len(shape)):
        if var.shape[i] == None:
            continue
        elif var.shape[i] != shape[i]:
            raise ValueError('Shape of minibatch and variable are incompatible')
    
    indices = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_indices.npy'))
    values = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_values.npy'))
    sparse_tensor = tf.sparse.SparseTensor(indices=indices, values=values, dense_shape=shape)
    var.assign(tf.cast(tf.sparse.to_dense(sparse_tensor), dtype=tf.float32))

In [3]:
var = tf.Variable(tf.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])

load_minibatch(0, var)

In [4]:
import os
import json

with open(os.path.join('DATA', 'numberize.json'), 'r') as f:
    numberize = json.load(f)
        
with open(os.path.join('DATA', 'reverse_numberize.json'), 'r') as f:
    reverse_numberize = json.load(f)

In [5]:
def verbalize_sentence(numberized_sentence):
    words = [reverse_numberize[str(word_number)] for word_number in numberized_sentence if word_number != 10001]
    return ' '.join(words)

def verbalize(vectorized_sentences):
    numberized_sentences = tf.argmax(vectorized_sentences, axis=2).numpy()
    return list(map(verbalize_sentence, numberized_sentences))

In [115]:
import numpy as np

class BatchGenerator(object):

    def __init__(self, num_batches, hidden_size):
        self.num_batches = num_batches
        self.current_idx = 0
        self.x = tf.Variable(np.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])
        
    def generate(self):
        while True:
            load_minibatch(self.current_idx, self.x)
            self.current_idx = (self.current_idx + 1) % self.num_batches
            X = np.stack([np.zeros([500, 10002])] + [self.x[:, i, :].numpy() for i in range(29)], axis=1)
            Y = [self.x[:, i, :].numpy() for i in range(30)]
            hidden_state_0 = np.zeros((500, hidden_size))
            cell_state_0 = np.zeros((500, hidden_size))
            yield [[X, hidden_state_0, cell_state_0], Y]
            
num_batches = 6806
batch_generator = BatchGenerator(6806, 128).generate()

In [7]:
from keras import Model, Input
from keras.layers import Dense, Activation, Dropout, LSTM, Lambda, Reshape
from keras.optimizers import Adam

sequence_length = 30
vocab_size = 10002
hidden_size = 128

Reshape_cell = Reshape((1, vocab_size)) 
LSTM_cell = LSTM(hidden_size, return_state = True)
Dense_cell = Dense(vocab_size, activation='softmax')

def LSTM_model(sequence_length, hidden_size, vocab_size):
    """
    Arguments:
    sequence_length -- length of input sequences
    hidden_size -- number of hidden units in LSTM cell
    vocab_size -- size of vocabulary in corpus 
    
    Returns:
    a keras model
    """
    
    # Define inputs to LSTM
    X = Input(shape=(sequence_length, vocab_size), name='sequence_input')
    hidden_state_0 = Input(shape=(hidden_size,), name='hidden_state_0')
    cell_state_0 = Input(shape=(hidden_size,), name='cell_state_0')
    
    hidden_state = hidden_state_0
    cell_state = cell_state_0
    outputs = []
    for t in range(sequence_length):
        # Select t-th time slice
        x = Lambda(lambda y: y[:, t, :])(X)
        # Rehapse and apply LSTM_cell to time slice
        x = Reshape_cell(x)
        hidden_state, _, cell_state = LSTM_cell(inputs=x, initial_state=[hidden_state, cell_state])
        # Apply Dense_cell to output
        output = Dense_cell(hidden_state)
        outputs.append(output)
        
    model = Model(inputs=[X, hidden_state_0, cell_state_0], outputs=outputs)
    
    return model

model = LSTM_model(sequence_length, hidden_size, vocab_size)
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Using TensorFlow backend.


In [52]:
batches_per_epoch = 6706
num_epochs = 1

model.fit_generator(batch_generator, batches_per_epoch, num_epochs)

Epoch 1/1
2/2 [==============================] - 46s 23s/step - loss: 93.9160 - dense_1_loss: 8.5459e-05 - dense_1_accuracy: 0.2000 - dense_1_accuracy_1: 0.1650 - dense_1_accuracy_2: 0.1830 - dense_1_accuracy_3: 0.1850 - dense_1_accuracy_4: 0.1920 - dense_1_accuracy_5: 0.1940 - dense_1_accuracy_6: 0.2170 - dense_1_accuracy_7: 0.2380 - dense_1_accuracy_8: 0.2600 - dense_1_accuracy_9: 0.2810 - dense_1_accuracy_10: 0.3230 - dense_1_accuracy_11: 0.3500 - dense_1_accuracy_12: 0.3900 - dense_1_accuracy_13: 0.4280 - dense_1_accuracy_14: 0.4780 - dense_1_accuracy_15: 0.5030 - dense_1_accuracy_16: 0.5510 - dense_1_accuracy_17: 0.5980 - dense_1_accuracy_18: 0.6430 - dense_1_accuracy_19: 0.6720 - dense_1_accuracy_20: 0.7220 - dense_1_accuracy_21: 0.7730 - dense_1_accuracy_22: 0.8080 - dense_1_accuracy_23: 0.8500 - dense_1_accuracy_24: 0.8860 - dense_1_accuracy_25: 0.9130 - dense_1_accuracy_26: 0.9460 - dense_1_accuracy_27: 0.9760 - dense_1_accuracy_28: 1.0000 - dense_1_accuracy_29: 1.0000


In [49]:
model.save_weights('model.h5')

In [ ]:
model.load_weights('model.h5')

In [95]:
def one_hot_random_sample(distribution):
    sample_index = tf.random.categorical(tf.math.log(distribution), 1)
    result = tf.one_hot(indices=sample_index, depth=vocab_size)
    return result

Random_sample_cell = Lambda(one_hot_random_sample)

def GeneratorModel(hidden_size, vocab_size, sequence_length):

    X = Input(shape=(1, vocab_size), name='initial_input')
    hidden_state_0 = Input(shape=(hidden_size,), name='hidden_state_0')
    cell_state_0 = Input(shape=(hidden_size,), name='cell_state_0')
    
    hidden_state = hidden_state_0
    cell_state = cell_state_0
    
    outputs = []
    x = Lambda(lambda y: y[:, 0, :])(X)
    x = Reshape_cell(x)
    hidden_state, _, cell_state = LSTM_cell(inputs=x, initial_state=[hidden_state, cell_state])
    distribution = Dense_cell2(hidden_state)
    result = Random_sample_cell(distribution)
    outputs.append(result)
    for t in range(1, sequence_length):
        hidden_state, _, cell_state = LSTM_cell(inputs=result, initial_state=[hidden_state, cell_state])
        distribution = Dense_cell2(hidden_state)
        result = Random_sample_cell(distribution)
        outputs.append(result)
        
    model = Model(inputs=[X, hidden_state_0, cell_state_0], outputs=outputs)
    
    return model

generator_model = GeneratorModel(hidden_size, vocab_size, sequence_length)

In [114]:
def decode(result):
    word_indices = [np.argmax(word_vector) for word_vector in result]
    sentence = verbalize_sentence(word_indices)
    return sentence

X = np.zeros((1,1,vocab_size))
hidden_state_0 = np.zeros((1, hidden_size))
cell_state_0 = np.zeros((1, hidden_size))

result = generator_model.predict([X, hidden_state_0, cell_state_0])
sentence = decode(result)
print(sentence)


I do so again .
